In [ ]:
# Method used:

# Hybrid approach using n-gram based guessing and high-frequency based guessing via trie data structure. 
# Also used multithreading.
import json
import random
import string
import secrets
import collections
import requests
import time
import re
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor


try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
# TRIE DATA STRUCTURE FOR FREQUENCY BASED GUESSING:

# Defines a Trie structure to efficiently store and search words, particularly when patterns with wildcard characters are invloved
# The search method allows for pattern matching while accommodating already guessed letters.
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search(self, pattern, guessed_letters):
        def dfs(node, index):
            if index == len(pattern):
                return node.is_end_of_word

            char = pattern[index]
            if char == '.':
                for child in node.children.values():
                    if dfs(child, index + 1):
                        return True
            elif char in node.children:
                return dfs(node.children[char], index + 1)
            return False

        return dfs(self.root, 0)

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        self.wrong_guesses = []
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location) 
        self.full_dictionary_common_letters = None
        self.current_dictionary = []
        self.trie = Trie()
        self.clean_word = None
        self.trigrms = {}  # Initialize trigrms
        self.letter_freq = collections.Counter()  # Initialize letter frequencie
        self.letter_set = sorted(set("".join(self.full_dictionary)))
        self.probs = [0] * len(self.letter_set)
        self.unigrm, self.bigrm, self.trigrm, self.fourgrm, self.fivegrm = self.make_grms(self.full_dictionary)
        self.left_tries = 6
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

 
        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link
## Implementing n-gram guessing techinique:

# To determine the best guess, the model uses n-gram counts.
#  N-grams are typically used to determine the likelihood of a word following another, but here it is used for letters.
# unigram , bigram , trigram , fourgram and fivegram used
    def nlp(self, word): 
       
        self.wrong_guesses = list(set(self.guessed_letters) - set(word))
        
        if len(self.guessed_letters) > 0 and self.guessed_letters[-1] in self.wrong_guesses and self.left_tries <4:
            self.update_grms()
        self.probs = [0] * len(self.letter_set)
        clean_word = word[::2]
        
        return self.fivegrm_probs(clean_word)

    def make_grms(self, dicti):
        unigrm = collections.defaultdict(lambda: collections.defaultdict(int))
        bi__grm = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
        tri__grm = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
        four__grm = collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int))))
        five__grm = collections.defaultdict(lambda: collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))))
        
        # go through each word in the dictionary
        for wrd in dicti:
            # check each letter in the dictionary and update the n-grm
            for j in range(len(wrd) - 4):
                bi__grm[len(wrd)][wrd[j]][wrd[j+1]] = bi__grm[len(wrd)][wrd[j]][wrd[j+1]] + 1
                tri__grm[wrd[j]][wrd[j+1]][wrd[j+2]] = tri__grm[wrd[j]][wrd[j+1]][wrd[j+2]]+ 1
                four__grm[wrd[j]][wrd[j+1]][wrd[j+2]][wrd[j+3]] =four__grm[wrd[j]][wrd[j+1]][wrd[j+2]][wrd[j+3]] + 1
                five__grm[wrd[j]][wrd[j+1]][wrd[j+2]][wrd[j+3]][wrd[j+4]] = five__grm[wrd[j]][wrd[j+1]][wrd[j+2]][wrd[j+3]][wrd[j+4]] +1
            j = len(wrd) - 4
            
            # filling left  n-grms for wrds which have small length
            if len(wrd) == 2:
                bi__grm[len(wrd)][wrd[0]][wrd[1]] += 1
            elif len(wrd) == 3:
                bi__grm[len(wrd)][wrd[0]][wrd[1]] += 1
                bi__grm[len(wrd)][wrd[1]][wrd[2]] += 1
                tri__grm[wrd[0]][wrd[1]][wrd[2]] += 1
                
            # filling out remaining
            elif len(wrd) >3:
                bi__grm[len(wrd)][wrd[j]][wrd[j+1]] += 1
                bi__grm[len(wrd)][wrd[j+1]][wrd[j+2]] += 1
                bi__grm[len(wrd)][wrd[j+2]][wrd[j+3]] += 1
                tri__grm[wrd[j]][wrd[j+1]][wrd[j+2]] += 1
                tri__grm[wrd[j+1]][wrd[j+2]][wrd[j+3]] += 1
                four__grm[wrd[j]][wrd[j+1]][wrd[j+2]][wrd[j+3]] += 1
            
            # filling the unigrms at last
            for letter in set(wrd):
                unigrm[len(wrd)][letter] += 1
        # returning the grms
        return unigrm, bi__grm, tri__grm, four__grm, five__grm
                    
        
    def update_grms(self):
        # update dictionary to remove incorrect guesses
        new_dict = [wrd for wrd in self.full_dictionary if not set(wrd).intersection(set(self.wrong_guesses))]
        self.unigrm, self.bigrm, self.trigrm, self.fourgrm, self.fivegrm = self.make_grms(new_dict)

    
    def fivegrm_probs(self, wrd):
        # gets prob of each letter in form of vector
        probs = [0] * len(self.letter_set)
        final_count = 0
        ltr_count = [0] * len(self.letter_set)

        for i in range(len(wrd) - 4):
                        
            # if (ltr ltr ltr _) is found
            if wrd[i] != '_' and wrd[i+1] != '_' and wrd[i+2] != '_' and wrd[i+3] != '_' and wrd[i+4] == '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+1]
                anchr_ltr_3 = wrd[i+2]
                anchr_ltr_4 = wrd[i+3]
                
                for j, letter in enumerate(self.letter_set):
                    if self.fivegrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4][letter] > 0 and letter not in self.guessed_letters:
                        final_count =final_count+ self.fivegrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4][letter]
                        ltr_count[j] =ltr_count[j] + self.fivegrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4][letter]
        
            # if (ltr ltr ltr _ ltr) is found
            elif wrd[i] != '_' and wrd[i+1] != '_' and wrd[i+2] != '_' and wrd[i+3] == '_' and wrd[i+4] != '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+1]
                anchr_ltr_3 = wrd[i+2]
                anchr_ltr_4 = wrd[i+4]
                
                for j, letter in enumerate(self.letter_set):
                    if self.fivegrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][letter][anchr_ltr_4] > 0 and letter not in self.guessed_letters:
                        final_count =final_count+ self.fivegrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][letter][anchr_ltr_4]
                        ltr_count[j] =ltr_count[j] + self.fivegrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][letter][anchr_ltr_4]
               
            # if (ltr ltr _ ltr ltr) is found
            elif wrd[i] != '_' and wrd[i+1] != '_' and wrd[i+2] == '_' and wrd[i+3] != '_' and wrd[i+4] != '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+1]
                anchr_ltr_3 = wrd[i+3]
                anchr_ltr_4 = wrd[i+4]
                
                
                for j, letter in enumerate(self.letter_set):
                    if self.fivegrm[anchr_ltr_1][anchr_ltr_2][letter][anchr_ltr_3][anchr_ltr_4] > 0 and letter not in self.guessed_letters:
                        final_count = final_count+ self.fivegrm[anchr_ltr_1][anchr_ltr_2][letter][anchr_ltr_3][anchr_ltr_4]
                        ltr_count[j] = ltr_count[j]+ self.fivegrm[anchr_ltr_1][anchr_ltr_2][letter][anchr_ltr_3][anchr_ltr_4]
               
            # if (ltr _ ltr ltr ltr) is found
            elif wrd[i] != '_' and wrd[i+1] == '_' and wrd[i+2] != '_' and wrd[i+3] != '_' and wrd[i+4] != '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+2]
                anchr_ltr_3 = wrd[i+3]
                anchr_ltr_4 = wrd[i+4]
                
                for j, letter in enumerate(self.letter_set):
                    if self.fivegrm[anchr_ltr_1][letter][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4] > 0 and letter not in self.guessed_letters:
                        final_count = final_count+ self.fivegrm[anchr_ltr_1][letter][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4]
                        ltr_count[j] = ltr_count[j]+ self.fivegrm[anchr_ltr_1][letter][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4]
        
            # if (_ ltr ltr ltr ltr) is found
            elif wrd[i] == '_' and wrd[i+1] != '_' and wrd[i+2] != '_' and wrd[i+3] != '_' and wrd[i+4] != '_':
                anchr_ltr_1 = wrd[i+1]
                anchr_ltr_2 = wrd[i+2]
                anchr_ltr_3 = wrd[i+3]
                anchr_ltr_4 = wrd[i+4]
                
                for j, letter in enumerate(self.letter_set):
                    if self.fivegrm[letter][anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4] > 0 and letter not in self.guessed_letters:
                        final_count =final_count+ self.fivegrm[letter][anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4]
                        ltr_count[j] =ltr_count[j]+ self.fivegrm[letter][anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][anchr_ltr_4]
        
        # gives prob of each letter
        if final_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = ltr_count[i] / final_count
        
        for i, p in enumerate(self.probs):
            self.probs[i] = p + probs[i] * (0.40)
        
        # return the value
        return self.fourgrm_probs(wrd)
    
    def fourgrm_probs(self, wrd):
                
        # gets prob of each letter in form of vector
        probs = [0] * len(self.letter_set)
        
        final_count = 0
        ltr_count = [0] * len(self.letter_set)

        # to find the patter
        for i in range(len(wrd) - 3):
                        
            #  "ltr ltr ltr _"
            if wrd[i] != '_' and wrd[i+1] != '_' and wrd[i+2] != '_' and wrd[i+3] == '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+1]
                anchr_ltr_3 = wrd[i+2]
                
                for j, letter in enumerate(self.letter_set):
                    if self.fourgrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][letter] > 0 and letter not in self.guessed_letters:
                        final_count =final_count+ self.fourgrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][letter]
                        ltr_count[j] =ltr_count[j]+ self.fourgrm[anchr_ltr_1][anchr_ltr_2][anchr_ltr_3][letter]
        
            # "ltr ltr _ ltr"
            elif wrd[i] != '_' and wrd[i+1] != '_' and wrd[i+2] == '_' and wrd[i+3] != '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+1]
                anchr_ltr_3 = wrd[i+3]
                
                for j, letter in enumerate(self.letter_set):
                    if self.fourgrm[anchr_ltr_1][anchr_ltr_2][letter][anchr_ltr_3] > 0 and letter not in self.guessed_letters:
                        final_count =final_count+ self.fourgrm[anchr_ltr_1][anchr_ltr_2][letter][anchr_ltr_3]
                        ltr_count[j] =ltr_count[j]+ self.fourgrm[anchr_ltr_1][anchr_ltr_2][letter][anchr_ltr_3]
               
            # "ltr _ ltr ltr"
            elif wrd[i] != '_' and wrd[i+1] == '_' and wrd[i+2] != '_' and wrd[i+3] != '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+2]
                anchr_ltr_3 = wrd[i+3]
                
            
                for j, letter in enumerate(self.letter_set):
                    if self.fourgrm[anchr_ltr_1][letter][anchr_ltr_2][anchr_ltr_3] > 0 and letter not in self.guessed_letters:
                        final_count =final_count+ self.fourgrm[anchr_ltr_1][letter][anchr_ltr_2][anchr_ltr_3]
                        ltr_count[j] = ltr_count[j]+ self.fourgrm[anchr_ltr_1][letter][anchr_ltr_2][anchr_ltr_3]
               
            # "_ ltr ltr ltr"
            elif wrd[i] == '_' and wrd[i+1] != '_' and wrd[i+2] != '_' and wrd[i+3] != '_':
                anchr_ltr_1 = wrd[i+1]
                anchr_ltr_2 = wrd[i+2]
                anchr_ltr_3 = wrd[i+3]
                
               
                for j, letter in enumerate(self.letter_set):
                    if self.fourgrm[letter][anchr_ltr_1][anchr_ltr_2][anchr_ltr_3] > 0 and letter not in self.guessed_letters:
                        final_count = final_count+ self.fourgrm[letter][anchr_ltr_1][anchr_ltr_2][anchr_ltr_3]
                        ltr_count[j] =ltr_count[j]+ self.fourgrm[letter][anchr_ltr_1][anchr_ltr_2][anchr_ltr_3]
        
        # calculating the prob of each letter
        if final_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = ltr_count[i] / final_count
        
        for i, p in enumerate(self.probs):
            self.probs[i] = p + probs[i] * (0.25)
        
        # return the value
        return self.trigrm_probs(wrd)

    def trigrm_probs(self, wrd):
            
        probs = [0] * len(self.letter_set)
        
        final_count = 0
        ltr_count = [0] * len(self.letter_set)

        for i in range(len(wrd) - 2):
                        
            if wrd[i] != '_' and wrd[i+1] != '_' and wrd[i+2] == '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+1]
                for j, letter in enumerate(self.letter_set):
                    if self.trigrm[anchr_ltr_1][anchr_ltr_2][letter] > 0 and letter not in self.guessed_letters:
                        final_count += self.trigrm[anchr_ltr_1][anchr_ltr_2][letter]
                        ltr_count[j] += self.trigrm[anchr_ltr_1][anchr_ltr_2][letter]
        
            elif wrd[i] != '_' and wrd[i+1] == '_' and wrd[i+2] != '_':
                anchr_ltr_1 = wrd[i]
                anchr_ltr_2 = wrd[i+2]
                

                for j, letter in enumerate(self.letter_set):
                    if self.trigrm[anchr_ltr_1][letter][anchr_ltr_2] > 0 and letter not in self.guessed_letters:
                        final_count += self.trigrm[anchr_ltr_1][letter][anchr_ltr_2]
                        ltr_count[j] += self.trigrm[anchr_ltr_1][letter][anchr_ltr_2]
               

            elif wrd[i] == '_' and wrd[i+1] != '_' and wrd[i+2] != '_':
                anchr_ltr_1 = wrd[i+1]
                anchr_ltr_2 = wrd[i+2]
                
                for j, letter in enumerate(self.letter_set):
                    if self.trigrm[letter][anchr_ltr_1][anchr_ltr_2] > 0 and letter not in self.guessed_letters:
                        final_count += self.trigrm[letter][anchr_ltr_1][anchr_ltr_2]
                        ltr_count[j] += self.trigrm[letter][anchr_ltr_1][anchr_ltr_2]
        
    
        if final_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = ltr_count[i] / final_count
        
    
        for i, p in enumerate(self.probs):
            self.probs[i] = p + probs[i] * (0.20)
        
        return self.bigrm_probs(wrd)
    
    
    def bigrm_probs(self, wrd):
    
        
        probs = [0] * len(self.letter_set)
        
        final_count = 0
        ltr_count = [0] * len(self.letter_set)
        
        for i in range(len(wrd) - 1):
            if wrd[i] != '_' and wrd[i+1] == '_':
                anchr_ltr = wrd[i]
                
                for j, letter in enumerate(self.letter_set):
                    if self.bigrm[len(wrd)][anchr_ltr][letter] > 0 and letter not in self.guessed_letters:
                        final_count += self.bigrm[len(wrd)][anchr_ltr][letter]
                        ltr_count[j] += self.bigrm[len(wrd)][anchr_ltr][letter]
                            
            elif wrd[i] == '_' and wrd[i+1]!= '_':
                anchr_ltr = wrd[i+1]
                
                for j, letter in enumerate(self.letter_set):
                    if self.bigrm[len(wrd)][letter][anchr_ltr] > 0 and letter not in self.guessed_letters:
                        final_count += self.bigrm[len(wrd)][letter][anchr_ltr]
                        ltr_count[j] += self.bigrm[len(wrd)][letter][anchr_ltr]
                                                                    
        if final_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = ltr_count[i] / final_count

        for i, p in enumerate(self.probs):
            self.probs[i] = p + probs[i] * (0.10)
        
        return self.unigrm_probs(wrd)
    
    
    def unigrm_probs(self, wrd):
    
                
        probs = [0] * len(self.letter_set)
        
        final_count = 0
        ltr_count = [0] * len(self.letter_set)
        
        for i in range(len(wrd)):
            if wrd[i] == '_':
                                
               for j, letter in enumerate(self.letter_set):
                    if self.unigrm[len(wrd)][letter] > 0 and letter not in self.guessed_letters:
                        final_count += self.unigrm[len(wrd)][letter]
                        ltr_count[j] += self.unigrm[len(wrd)][letter]
                       

        if final_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = ltr_count[i] / final_count
                
        for i, p in enumerate(self.probs):
            self.probs[i] = p + probs[i] * (0.05)
        
        final_probs = [0] * len(self.letter_set)
        if sum(self.probs) > 0:
            for i in range(len(self.probs)):
                final_probs[i] = self.probs[i] / sum(self.probs)
            
        self.probs = final_probs
        
        max_prob = 0
        guess_letter = ''
        for i, letter in enumerate(self.letter_set):
            if self.probs[i] > max_prob:
                max_prob = self.probs[i]
                guess_letter = letter
        
        if guess_letter == '':
            letters = self.letter_set.copy()
            random.shuffle(letters)
            letters_s = ['e','a','i','o','u'] + letters
            for letter in letters_s:
                if letter not in self.guessed_letters:
                    return letter
            
        return guess_letter

    ## frequency based guess is made
    ## tri data structure used to optimize it and to reduce api calls

    def frequency_guess(self, word):
        if not self.clean_word:
            self.clean_word = word.replace(" ", "").replace("_", ".")
        len_word = len(self.clean_word)

        filtered_words = [word for word in self.current_dictionary if len(word) == len_word and self.trie.search(self.clean_word, self.guessed_letters)]

        if not self.full_dictionary_common_letters:
            letter_freq = defaultdict(int)
            for word in self.current_dictionary:
                for char in word:
                    if char not in self.guessed_letters:
                        letter_freq[char] += 1
            self.full_dictionary_common_letters = sorted(letter_freq.items(), key=lambda x: x[1], reverse=True)

        for letter, instance_count in self.full_dictionary_common_letters:
            if letter not in self.guessed_letters:
                return letter

        return '!'
    
    ## GUESS FUNCTION IS USED TO GUESS A NEW LETTER
    # both of the approaches are used depending on number of tries 
    def guess(self, word, tries):
        frequency_result = self.frequency_guess(word)
        if tries<3 :
            nlp_result = self.nlp(word)
    
            return nlp_result
        else:
            return frequency_result
    
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.wrong_guesses = []
        self.probs = [0] * len(self.letter_set)
        self.left_tries = 6
        self.update_grms()
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        if not self.full_dictionary:
            print(self.full_dictionary)
            self.full_dictionary = self.build_dictionary(self.full_dictionary_location)
            for word in self.full_dictionary:
                self.trie.insert(word)
        self.clean_word = None
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word,tries_remains)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)
        
## implementing the game by calling start_game function after making the object of it
api = HangmanAPI(access_token="f82dec5fd11225cac40c7c1b3e086e", timeout=2000)
for i in range(1000):
    print('Playing ', i, ' th game')
    api.start_game(practice=0,verbose=False)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    success_rate = total_recorded_successes/total_recorded_runs
    print('overall success rate = %.3f' % success_rate)
    time.sleep(0.5)
